# Inférence
Dans cette leçon, vous allez inférer des sentiments et des sujets à partir d'avis sur des produits et d'articles d'actualité.

## Configuration

In [ ]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)


In [ ]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0,
    )
    return response.choices[0].message.content

## Texte de revue de produit

In [ ]:
lamp_review = """
J'avais besoin d'une belle lampe pour ma chambre, 
et celle-ci offrait un espace de rangement supplémentaire 
à un prix raisonnable. Elle est arrivée rapidement.
Le cordon de la lampe s'est cassé pendant le transport et 
l'entreprise a rapidement envoyé un nouveau cordon, qui 
est également arrivé en quelques jours. C'était facile à 
assembler. J'avais une pièce manquante, alors j'ai contacté 
leur support et ils m'ont très rapidement envoyé la pièce 
manquante ! Lumina me semble être une excellente entreprise 
qui se soucie de ses clients et de ses produits !
"""

## Sentiment (positif/negatif)

In [ ]:
prompt = f"""
Quel est le sentiment de l'avis sur le produit suivant, 
délimité par des triples accents graves ?

Texte de l'avis : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Quel est le sentiment de l'avis sur le produit suivant,
délimité par des triples accents graves ?

Donnez votre réponse en un seul mot, soit "positif"
soit "négatif".

Texte de l'avis : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Identifier le type d'émotions

In [ ]:
prompt = f"""
Identifiez une liste d'émotions que l'auteur de l'avis 
suivant exprime. Incluez au maximum cinq éléments dans la 
liste. Formatez votre réponse sous forme de liste de mots 
en minuscules séparés par des virgules.

Texte de l'avis : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Identifier la colère

In [ ]:
prompt = f"""
L'auteur de l'avis suivant exprime-t-il de la colère ? 
L'avis est délimité par des triples accents graves.
Donnez votre réponse par "oui" ou "non".

Texte de l'avis : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Extraire le nom du produit et le nom de l'entreprise à partir des avis des clients.

In [ ]:
prompt = f"""
Identifiez les éléments suivants à partir du texte de l'avis :
- Article acheté par l'évaluateur
- Entreprise qui a fabriqué l'article

L'avis est délimité par des triples accents graves.
Formattez votre réponse en tant qu'objet JSON avec "Item" et "Brand" comme clés.
Si l'information n'est pas disponible, utilisez "unknown" comme valeur.
Rendez votre réponse aussi courte que possible.

Texte de l'avis : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Faire plusieurs tâche à la fois

In [ ]:
prompt = f"""
Identifiez les éléments suivants à partir du texte de l'avis :
- Sentiment (positif ou négatif)
- L'évaluateur exprime-t-il de la colère ? (vrai ou faux)
- Article acheté par l'évaluateur
- Entreprise qui a fabriqué l'article

L'avis est délimité par des triples accents graves.
Formattez votre réponse en tant qu'objet JSON avec "Sentiment", "Anger", "Item" et "Brand" comme clés.
Si l'information n'est pas disponible, utilisez "unknown" comme valeur.
Rendez votre réponse aussi courte que possible.
Formatez la valeur Anger en tant que booléen.

Texte de l'avis : '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

## Déduire les thèmes abordés dans un article

In [ ]:
story = """
Dans une récente enquête menée par le gouvernement, les employés du 
secteur public ont été invités à évaluer leur niveau de satisfaction 
vis-à-vis du département dans lequel ils travaillent. Les résultats 
ont révélé que la NASA était le département le plus populaire avec un 
taux de satisfaction de 95 %.

Un employé de la NASA, John Smith, a commenté les résultats, déclarant : 
« Je ne suis pas surpris que la NASA soit arrivée en tête. C'est un excellent 
endroit pour travailler avec des personnes incroyables et des opportunités 
formidables. Je suis fier de faire partie d'une organisation aussi innovante. »

Les résultats ont également été bien accueillis par l'équipe de direction de 
la NASA, le directeur Tom Johnson déclarant : « Nous sommes ravis d'apprendre 
que nos employés sont satisfaits de leur travail à la NASA. Nous avons une équipe 
talentueuse et dévouée qui travaille sans relâche pour atteindre nos objectifs, 
et c'est fantastique de voir que leur dur travail porte ses fruits. »

L'enquête a également révélé que l'Administration de la sécurité sociale avait 
le taux de satisfaction le plus bas, avec seulement 45 % des employés indiquant 
qu'ils étaient satisfaits de leur travail. Le gouvernement s'est engagé à répondre 
aux préoccupations soulevées par les employés dans l'enquête et à œuvrer pour améliorer 
la satisfaction au travail dans tous les départements.
"""

## Déduire 5 thèmes

In [ ]:
prompt = f"""
Déterminez cinq sujets qui sont discutés dans le texte suivant, 
qui est délimité par des triples accents graves.

Faites en sorte que chaque élément ait une ou deux mots de long.

Formatez votre réponse sous forme d'une liste d'éléments séparés 
par des virgules.

Exemple de texte :'''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
response.split(sep=',')

In [ ]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Création d'alertes pour certains sujets

In [ ]:
prompt = f"""
Déterminez si chaque élément de la liste suivante de sujets est un sujet 
du texte ci-dessous, qui est délimité par des triples backticks.
Ne pas introduire la liste par une phrase d'introduction.

Donnez votre réponse sous forme de liste avec 0 ou 1 pour chaque sujet.
Répondre sous la forme : <sujet> : <0 ou 1>

Liste de sujets : {", ".join(topic_list)}

Exemple de texte : '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
response.split(sep='\n')

In [ ]:
topic_dict = {i.split(' : ')[0]: int(i.split(' : ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERTE: Nouveau sujet sur la NASA!")

## A vous de jouer !